<a href="https://colab.research.google.com/github/Adiidil2005/-LLM-powered-platform-to-recommend-optimal-sustainable-groceries-/blob/main/ibm_project_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import google.generativeai as genai
from google.colab import userdata

try:
    API_KEY = userdata.get('GEMINI_API_KEY')
    if not API_KEY:
        raise ValueError("GEMINI_API_KEY not found in Colab user data.")
    genai.configure(api_key=API_KEY)
except Exception as e:
    print(f"Error configuring Gemini API: {e}")
    print("Please store your Gemini API key in Colab's user data secrets named 'GEMINI_API_KEY'.")

from google.colab import files

uploaded = files.upload()

if uploaded:
    image_name = list(uploaded.keys())[0]
    image_content = uploaded[image_name]
    print(f"File '{image_name}' uploaded successfully.")
else:
    print("No file uploaded.")

prompt = """
Extract the tabular data from this image.
Present the data in a structured format, such as JSON or a delimited string,
where each row is a record and each column is a field.
Identify the headers of the table if present.
"""

model = genai.GenerativeModel('gemini-1.5-flash')

if uploaded:
    image_name = list(uploaded.keys())[0]
    image_content = uploaded[image_name]
    image_parts = [
        {
            "mime_type": f"image/{image_name.split('.')[-1]}",
            "data": image_content
        }
    ]

    response = model.generate_content([prompt, image_parts[0]])
    extracted_data = response.text
    print("Extracted data:")
    print(extracted_data)
else:
    extracted_data = None
    print("No image uploaded to extract data from.")
import pandas as pd
import pandas as pd
import json
import re

if extracted_data:
    try:
        # Extract the JSON block from ```json ... ``` using regex
        json_match = re.search(r"```json\s*(.*?)\s*```", extracted_data, re.DOTALL)
        if not json_match:
            raise ValueError("No valid JSON code block found.")

        json_string = json_match.group(1).strip()
        parsed = json.loads(json_string)

        # Check if the parsed data is a list of dictionaries
        if isinstance(parsed, list) and all(isinstance(item, dict) for item in parsed):
            df = pd.DataFrame(parsed)
        elif isinstance(parsed, dict):
            # Extract headers and data if the JSON is a dictionary with 'headers' and 'data'
            headers = parsed.get("headers", [])
            data = parsed.get("data", []) # Changed from "rows" back to "data"

            normalized_data = []
            for row in data:
                normalized_row = {key: row.get(key, None) for key in headers}
                normalized_data.append(normalized_row)

            df = pd.DataFrame(normalized_data)
        else:
            raise ValueError("JSON data is not in the expected list or dictionary format.")

        print("\n✅ Final DataFrame:")
        display(df)

    except json.JSONDecodeError as e:
        print(f"\n❌ JSON decode error: {e}")
        df = None
    except Exception as e:
        print(f"\n❌ Unexpected error: {e}")
        df = None
else:
    print("No data extracted from the image.")
    df = None

Saving costco-receipt-template.png.jpg to costco-receipt-template.png.jpg
File 'costco-receipt-template.png.jpg' uploaded successfully.
Extracted data:
```json
[
  {
    "Item Code": "1358822",
    "Item Name": "CHARMIN",
    "Price": "22.49",
    "Modifier": "A"
  },
  {
    "Item Code": "9462",
    "Item Name": "ORANGES",
    "Price": "12.99",
    "Modifier": ""
  },
  {
    "Item Code": "83424",
    "Item Name": "HONEYDEW",
    "Price": "3.99",
    "Modifier": ""
  },
  {
    "Item Code": "8789",
    "Item Name": "GOLDEN PINE",
    "Price": "2.99",
    "Modifier": ""
  },
  {
    "Item Code": "660212",
    "Item Name": "DOVE16CT",
    "Price": "15.99",
    "Modifier": "A"
  },
  {
    "Item Code": "949985",
    "Item Name": "ADIDAS SOCK",
    "Price": "12.99",
    "Modifier": "A"
  },
  {
    "Item Code": "949985",
    "Item Name": "ADIDAS SOCK",
    "Price": "12.99",
    "Modifier": "A"
  },
  {
    "Item Code": "879104",
    "Item Name": "SOUR DOUGH",
    "Price": "6.99",
    "Mod

,Item Code,Item Name,Price,Modifier
0,1358822,CHARMIN,22.49,A
1,9462,ORANGES,12.99,
2,83424,HONEYDEW,3.99,
3,8789,GOLDEN PINE,2.99,
4,660212,DOVE16CT,15.99,A
5,949985,ADIDAS SOCK,12.99,A
6,949985,ADIDAS SOCK,12.99,A
7,879104,SOUR DOUGH,6.99,
8,858187,OATNUT BREAD,5.79,
9,31865,BAGELS,5.99,


In [ ]:
import google.generativeai as genai

if df is not None:
    table_text = df.fillna("").to_csv(index=False)

    prompt = f"""
You are a smart shopping assistant.
Given this table of products that customers have bought:

{table_text}

For each item, suggest alternative or complementary products that the customer might consider.
Provide the alternative only incase the item can be replaced by a more eco friendly product
Provide your answer in a structured bullet-point format grouped by row/product.
"""
    alt_model = genai.GenerativeModel('gemini-1.5-flash')

    alt_response = alt_model.generate_content(prompt)

    print("\n🧠 Suggested Alternatives:")
    print(alt_response.text)
else:
    print("DataFrame is empty. Cannot generate alternative suggestions.")



🧠 Suggested Alternatives:
* **CHARMIN (1358822):**  Consider a more sustainable toilet paper brand made from recycled materials or bamboo.  Many brands offer comparable quality with a lower environmental impact.

* **ORANGES (9462):**  Consider other citrus fruits like mandarins, grapefruit, or lemons depending on intended use.

* **HONEYDEW (83424):**  Consider other melons like cantaloupe or watermelon, depending on preference.

* **GOLDEN PINE (8789):**  Consider other types of pineapples or other tropical fruits depending on the recipe or intended use.

* **DOVE16CT (660212):** Consider a bar soap instead to reduce plastic waste, or look for Dove products with more sustainable packaging.

* **ADIDAS SOCK (949985):**  No alternative suggested.  This is a specific item. Consider checking for more sustainable materials (like organic cotton) if you are focused on this aspect.

* **ADIDAS SOCK (949985):**  No alternative suggested. Same as above.

* **SOUR DOUGH (879104):** Consider ot